In [ ]:
import pandas as pd

data_final_test = 'data_final_test.csv'
data_final_train = 'data_final_train.csv'

# Read the Excel file
data_final_test = pd.read_csv(data_final_test)
data_final_train = pd.read_csv(data_final_train)

In [ ]:
pd.set_option('display.max_colwidth', None)
data_final_train

In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

In [ ]:
data = Dataset.from_pandas(data_final_train)
eval_dataset = Dataset.from_pandas(data_final_test)

In [ ]:
model_id="llama-2-13b-chat-hf"
use_flash_attention = False
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    use_cache=False,
    use_flash_attention_2=use_flash_attention,
    device_map="auto",
    torch_dtype=torch.float16
)


print(model)

In [ ]:
model.config.pretraining_tp = 1

In [ ]:
# LoRA config based on QLoRA paper
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]
)
# Prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [ ]:
from transformers import TrainingArguments

training_arguments = TrainingArguments(
    output_dir="LLaMa2_13B_Chat-finetuned-New_annotations_v10",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=100,
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    # max_grad_norm=0.3,
    # warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    disable_tqdm=False,
    push_to_hub=True
)

In [ ]:
from trl import SFTTrainer


trainer = SFTTrainer(
    model=model,
    train_dataset=data,
    # eval_dataset=eval_dataset,
    dataset_text_field="text",
    peft_config=peft_config,
    max_seq_length=2048,
    tokenizer=tokenizer,
    packing=True,
    # formatting_func=format_prompt,
    args=training_arguments,
)

In [ ]:
refined_model = "LLaMa2_13B_Chat-finetuned-New_annotations_v10"
# Train
trainer.train()
# Save model

In [ ]:
trainer.model.save_pretrained(refined_model)

In [ ]:
#Inferencing

In [ ]:
# incorporating our adapter weights into the base model.

In [ ]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    training_arguments.output_dir,
    low_cpu_mem_usage=True,
)

# Merge LoRA and base model
merged_model = model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_LLaMa2_13B_Chat-finetuned_New_annotations_v10_vllm", safe_serialization=True)
tokenizer.save_pretrained("merged_LLaMa2_13B_Chat-finetuned_New_annotations_v10_vllm")